In [1]:
from bokeh import plotting as plt
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
import pathlib
mcnp_dir = pathlib.Path.cwd() / "mcnp"

# Load secrets

In [3]:
from plumbum import cmd

lookup_secret = lambda field: cmd.secret_tool("lookup", "app", "phymat", "field", field).strip()
user = lookup_secret("user")
domain = lookup_secret("domain")
port = lookup_secret("port")
password = lookup_secret("password")

# Start SSHFS

In [4]:
import pexpect
p = pexpect.spawn('bash')
p.sendline(f'sshfs {user}@{domain}:mcnp {mcnp_dir} -p {port}')
p.expect("password:")
p.sendline(password)
p.sendline('exit')
p.expect_exact(pexpect.EOF)

0

# Define SSH session

In [5]:
from plumbum.machines.paramiko_machine import ParamikoMachine
from plumbum.commands import BaseCommand
from proxy import Proxy
    
class RemoteCommandProxy(Proxy):
    proxy_cls = BaseCommand
    
    def __call__(self, *args, _kwargs=None, **kwargs):
        new_args = list(args)
        new_args.extend(f'{k}={v}' for k, v in kwargs.items())
        if _kwargs is None:
            _kwargs = {}
        return object.__getattribute__(self, '_obj')(*new_args, _kwargs)
        
    
rem = ParamikoMachine(domain, user, port, password, keep_alive=30)
rem._cwd = rem.cwd.chdir('mcnp')
remote = RemoteCommandProxy(rem)

In [6]:
clean =  remote['./clean.sh']
mcnp = remote['./mcnp']

In [7]:
from contextlib import contextmanager
from plumbum import ProcessExecutionError

@contextmanager
def ignore_execution_error():
    try:
        yield
    except ProcessExecutionError:
        pass

# Part 1

In [8]:
%%writefile mcnp/pt1.ip
MESSAGE:

Practical Monte Carlo part 1.
C
C Cells
1 1 7.92 1 -2 3 -4 5 -6 (-7:8:-9:10:-11) $ Walls
2 0 (-1:2:-3:4:-5:6) $ Void
3 2 1.0 7 -8 9 -10 11 -12 $ Water
4 0 7 -8 9 -10 12 -6 $ Air gap (void)

C Surfaces
C Define outer walls
1 PX -5.20
2 PX 5.20
3 PY -10.20
4 PY 10.20
5 PZ 0.0
6 PZ 20.20
C Define XY wall surfaces
7 PX -5.00
8 PX 5.00
9 PY -10.0
10 PY 10.0
C Define Z wall / water surfaces
11 PZ 0.20
12 PZ 19.20

C Cell importance MAP
IMP:N 1 0 1 1         $  s
M1   26000.42c 0.74 24000.42c 0.18 28000.42c 0.08 $ Stainless steel
M2   1001.42c 0.6667  8016.42c 0.3333   $ Pure water
C Tallying energy fluence (nX where X is type (2) and n an ID: {1, 2, ...})
F12:N (1 2)
F22:N (3 4)
C Tally energy bins for all tallies (log 10 space)
E0 1E-9 1E-8 1E-7 1E-6 1E-5 1E-4 1E-3 1E-2 1E-1 1 10
C MC type
MODE N
C Thermal neutron induced fission of 235U at (0,0,20mm)
SDEF POS=0.0 0.0 2.0 ERG=D1
SP1  -3 0.988 2.249
C NUMBER OF PARTICLE HISTORIES TO RUN
NPS  20000
PRDMP 0 0 1 1 0

Overwriting mcnp/pt1.inp


In [ ]:
with ignore_execution_error():
    clean()
mcnp(inp="pt1.ip")

In [9]:
from mctal import tokenizer
tokens = [*tokenizer.tokenize_file("mcnp/mctal")]

In [10]:
from derp import parse
from mctal import g
mctal = next(iter(parse(g.mctal, tokens)))
for tally in mctal.tallies:
    print(tally.data)

((0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (7.56103e-08, 1.0), (0.0, 0.0), (6.22431e-08, 1.0), (1.37853e-07, 0.7104))
((0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0))


# Part 3

In [13]:
%%writefile mcnp/pt3.ip
MESSAGE:

Practical Monte Carlo part 3.
C
C Cells
1 1 7.92 1 -2 3 -4 5 -6 (-7:8:-9:10:-11) $ Walls
2 0 (-1:2:-3:4:-5:6)  (-13:14:-1:2:-3:4) $ Void
3 2 1.0 7 -8 9 -10 11 -12 $ Water
4 0 7 -8 9 -10 12 -6 $ Air gap (void)
5 3 2.3 -5 13 $ Concrete floor

C Surfaces (100x100x60 bucket with 2mm wall thickness)
C Define outer walls
1 PX -100.20
2 PX 100.20
3 PY -100.20
4 PY 100.20
5 PZ 0.0
6 PZ 100.20
C Define XY wall surfaces
7 PX -100.00
8 PX 100.00
9 PY -100.0
10 PY 100.0
C Define Z wall / water surfaces
11 PZ 0.20
12 PZ 98.20 $ Water surface
13 PZ -304.8 $ Concrete floor lower surface

C Cell importance MAP
IMP:N 1 1 1 1         $  s
M1   26000.42c 0.74 24000.42c 0.18 28000.42c 0.08 $ Stainless steel
M2   1001.42c 0.6667  8016.42c 0.3333   $ Pure water
M3  8000.42c 0.53 14000.42c 0.34 20000.42c 0.10 1000.42c 0.03
C Tallying energy fluence (nX where X is type (2) and n an ID: {1, 2, ...})
F12:N 1 2 $ Located at Z=60.20 + 50
C Tally energy bins for all tallies (log 10 space)
E0 1E-9 1E-8 1E-7 1E-6 1E-5 1E-4 1E-3 1E-2 1E-1 1 10
C MC type
MODE N
C Thermal neutron induced fission of 235U at (0,0,20mm)
SDEF POS=0.0 0.0 2.0 ERG=D1
SP1  -3 0.988 2.249
C NUMBER OF PARTICLE HISTORIES TO RUN
NPS  200000
PRDMP 0 0 1 1 0


Writing mcnp/pt3.ip
